# Load data using `read_csv`

In [13]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x : '%.3f' % x)
pd.set_option('max_columns', None)

In [14]:
df = pd.read_csv("../inflearn_pandas_part1_material/my_data/naver_finance/2016_12.csv")

In [15]:
df.head()

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.310,7.313,4.563,1333.223,12.193,5.091,15.515,5436.413,48112.402,220556.160,10.301,1.164,0.254,56000.000,68500.000
1,BGF,860.773,9.315,214.481,1846.192,21.624,10.433,8.015,3703.577,18648.623,1737.263,22.757,4.519,48.514,42140.000,15250.000
2,BNK금융지주,49126.760,14.499,10.546,5181.144,7.919,0.564,NaN,1568.237,20810.660,15358.993,5.535,0.417,0.565,8680.000,9420.000
3,BYC,2118.576,7.625,8.281,175.433,4.543,2.463,10.748,20872.312,471887.000,252211.450,19.020,0.841,1.574,397000.000,306000.000
4,CJ,239541.970,5.230,2.379,5698.234,6.080,2.253,5.158,6257.152,114276.080,672045.900,28.181,1.543,0.262,176334.000,171148.000


### 수익률 구하기 (16.12 ~ 17.12)

In [16]:
df['rtn'] = df['price2'] / df['price'] - 1

## 2. Give group-number(or score) according to PER

### non-equal length partition

#### boolean indexing & loc 사용

- 뒤에 Grouping by continuous variables section에서는 더 쉽게 가능

In [22]:
bound1 = df['PER(배)'] >= 10
bound2 = (5 <= df['PER(배)']) & (df['PER(배)'] < 10)
bound3 = (0 <= df['PER(배)']) & (df['PER(배)'] < 5) 
bound4 = df['PER(배)'] < 0

In [23]:
df.loc[bound1, 'PER_Score'] = 1
df.loc[bound2, 'PER_Score'] = 2
df.loc[bound3, 'PER_Score'] = 3
df.loc[bound4, 'PER_Score'] = -1

In [24]:
df['PER_Score'].head()

0   1.000
1   1.000
2   2.000
3   1.000
4   1.000
Name: PER_Score, dtype: float64

In [25]:
df['PER_Score'].nunique()

4

In [26]:
df['PER_Score'].value_counts()

1.000     378
2.000     148
-1.000    120
3.000      23
Name: PER_Score, dtype: int64

In [27]:
df['PER_Score'].isna().sum()

12

### boolean series의 연산 특성 사용

In [28]:
df.loc[:, "PER_Score1"] = (bound1 * 1) + (bound2 * 2) + (bound3 * 3) + (bound4 * -1)

In [29]:
df['PER_Score1'].value_counts()

 1    378
 2    148
-1    120
 3     23
 0     12
Name: PER_Score1, dtype: int64

In [31]:
df['PER_Score1'].head()

0    1
1    1
2    2
3    1
4    1
Name: PER_Score1, dtype: int64

### 위의 두 score series는 서로 같을까?

In [32]:
df['PER_Score'].equals(df['PER_Score1'])

False

In [33]:
df['PER_Score'].dtypes
df['PER_Score1'].dtypes

dtype('float64')

dtype('int64')

### 2.1.4. `cut()`

In [37]:
per_cuts = pd.cut(
    df['PER(배)'],
    [-np.inf, 0,5,10,np.inf],
)

per_cuts.head()

0    (10.0, inf]
1    (10.0, inf]
2    (5.0, 10.0]
3    (10.0, inf]
4    (10.0, inf]
Name: PER(배), dtype: category
Categories (4, interval[float64]): [(-inf, 0.0] < (0.0, 5.0] < (5.0, 10.0] < (10.0, inf]]

In [39]:
per_cuts.isna().sum()

12

In [40]:
per_cuts.value_counts()

(10.0, inf]    378
(5.0, 10.0]    148
(-inf, 0.0]    120
(0.0, 5.0]      23
Name: PER(배), dtype: int64

In [44]:
bins = [-np.inf, 10,20,np.inf]
# 1. -np.inf ~ 10
# 2. 10 ~ 20
# 3. 20 ~ np.inf
labels = ["저평가주","보통주","고평가주"]
per_cuts2 = pd.cut(
    df['PER(배)'],
    bins=bins,
    labels=labels
)
per_cuts2.head()

0     보통주
1    고평가주
2    저평가주
3     보통주
4    고평가주
Name: PER(배), dtype: category
Categories (3, object): ['저평가주' < '보통주' < '고평가주']

## 3. SAME number of members in each group

### qcut

In [46]:
df.loc[:, 'PER_Score2'] = pd.qcut(df['PER(배)'], 3)

In [48]:
df['PER_Score2'].value_counts()

(-27857.497, 7.666]    223
(7.666, 16.495]        223
(16.495, 7056.129]     223
Name: PER_Score2, dtype: int64

In [52]:
df.loc[:, 'PER_Score2'] = pd.qcut(df['PER(배)'], 10, labels=range(1,11))

In [53]:
df['PER_Score2'].value_counts()

1     67
2     67
3     67
4     67
5     67
7     67
8     67
9     67
10    67
6     66
Name: PER_Score2, dtype: int64

In [54]:
df['PER_Score2'].hasnans

True

In [55]:
df['PER_Score2'].isna().sum()

12

In [58]:
df = df.dropna(subset=['PER_Score2'])

In [59]:
df['PER_Score2'].isna().sum()

0

## 4. Split - Apply - Combine

In [61]:
df = pd.read_csv("../inflearn_pandas_part1_material/my_data/naver_finance/2016_12.csv")
df.shape

(681, 16)

In [62]:
df = df.dropna()
df.shape

(609, 16)

In [63]:
g_df = df.copy()
g_df.head()

,ticker,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2
0,AK홀딩스,29218.310,7.313,4.563,1333.223,12.193,5.091,15.515,5436.413,48112.402,220556.160,10.301,1.164,0.254,56000.000,68500.000
1,BGF,860.773,9.315,214.481,1846.192,21.624,10.433,8.015,3703.577,18648.623,1737.263,22.757,4.519,48.514,42140.000,15250.000
3,BYC,2118.576,7.625,8.281,175.433,4.543,2.463,10.748,20872.312,471887.000,252211.450,19.020,0.841,1.574,397000.000,306000.000
4,CJ,239541.970,5.230,2.379,5698.234,6.080,2.253,5.158,6257.152,114276.080,672045.900,28.181,1.543,0.262,176334.000,171148.000
5,CJ CGV,14322.454,4.911,0.393,56.217,3.105,0.284,2.919,583.051,17654.154,67682.260,120.744,3.988,1.040,70400.000,74200.000


## 4.1. Some score assignment

In [64]:
g_df['rtn'] = g_df['price2'] / g_df['price'] -1 

In [65]:
g_df.loc[:, 'PER_score'] = pd.qcut(g_df['PER(배)'],10,labels=range(1,11))
g_df.loc[:, 'PBR_score'] = pd.qcut(g_df['PBR(배)'], 10, labels=range(1,11))

In [66]:
g_df.set_index('ticker', inplace=True)

In [67]:
g_df.head()

,매출액(억원),영업이익률(%),순이익률(%),당기순이익(억원),ROE(%),ROA(%),ROIC(%),EPS(원),BPS(원),SPS(원),PER(배),PBR(배),PSR(배),price,price2,rtn,PER_score,PBR_score
ticker,,,,,,,,,,,,,,,,,,
AK홀딩스,29218.310,7.313,4.563,1333.223,12.193,5.091,15.515,5436.413,48112.402,220556.160,10.301,1.164,0.254,56000.000,68500.000,0.223,5,7
BGF,860.773,9.315,214.481,1846.192,21.624,10.433,8.015,3703.577,18648.623,1737.263,22.757,4.519,48.514,42140.000,15250.000,-0.638,8,10
BYC,2118.576,7.625,8.281,175.433,4.543,2.463,10.748,20872.312,471887.000,252211.450,19.020,0.841,1.574,397000.000,306000.000,-0.229,8,5
CJ,239541.970,5.230,2.379,5698.234,6.080,2.253,5.158,6257.152,114276.080,672045.900,28.181,1.543,0.262,176334.000,171148.000,-0.029,9,8
CJ CGV,14322.454,4.911,0.393,56.217,3.105,0.284,2.919,583.051,17654.154,67682.260,120.744,3.988,1.040,70400.000,74200.000,0.054,10,10


## 4.2. groupby() & aggregation

- `groupby()`
  - 실제로 grouping까지는 하지 않고, grouping이 가능한지 validation만 진행(preparation)
- `Aggregation`
  - 2가지 요소로 구성
    - appregating columns
    - aggregatin functions
      - e.g. `sum, min, max, mean, count, variacne, std` etc

- 결국 3가지 요소만 충족시키면 됨
  - Grouping columns (categorial data type)
  - Aggregating columns
  - Aggregating functions

### 4.2.1. Examining the groupby object

In [68]:
g_df.groupby('PER_score')

In [74]:
g_df_obj = g_df.groupby(["PBR_score", "PER_score"])
g_df_obj
# Grouping 가능

In [70]:
type(g_df_obj)

pandas.core.groupby.generic.DataFrameGroupBy

In [72]:
g_df_obj.ngroups

96

In [75]:
g_df['PBR_score'].nunique()
g_df['PER_score'].nunique()

10

10

### 4.2.2. aggregation

- 반드시 "aggregationg" 기능이 있는 funciton을 써야함
  - min,max,mean,median,sum,var,size,nunique,idxmax

In [87]:
g_df.groupby("PBR_score").agg(
    {
        "rtn":"mean" # = np.mean
    }
)
# PBR_score로 Grouping하고 rtn의 평균값을 도출

,rtn
PBR_score,
1,-0.001
2,0.020
3,-0.021
4,0.161
5,-0.012
6,-0.043
7,0.150
8,0.058
9,0.139


In [89]:
# 같은 방법
g_df.groupby('PBR_score')['rtn'].mean().head()
g_df.groupby('PBR_score')[['rtn']].mean().head()
g_df.groupby('PBR_score')[['rtn']].agg("mean").head()

PBR_score
1   -0.001
2    0.020
3   -0.021
4    0.161
5   -0.012
Name: rtn, dtype: float64

,rtn
PBR_score,
1,-0.001
2,0.020
3,-0.021
4,0.161
5,-0.012


,rtn
PBR_score,
1,-0.001
2,0.020
3,-0.021
4,0.161
5,-0.012


In [90]:
g_df.groupby('PBR_score')[['rtn','PBR(배)']].agg(["mean","std"]).head()

rtn       PBR(배)      
            mean   std   mean   std
PBR_score                          
1         -0.001 0.262  0.355 0.121
2          0.020 0.280  0.519 0.029
3         -0.021 0.219  0.627 0.034
4          0.161 0.770  0.751 0.040
5         -0.012 0.265  0.889 0.043

In [92]:
g_df.groupby("PBR_score").agg(
    {
        "rtn":["mean","std"],
        "PBR(배)" : "mean"
    }
)

rtn       PBR(배)
            mean   std   mean
PBR_score                    
1         -0.001 0.262  0.355
2          0.020 0.280  0.519
3         -0.021 0.219  0.627
4          0.161 0.770  0.751
5         -0.012 0.265  0.889
6         -0.043 0.251  1.056
7          0.150 0.704  1.222
8          0.058 0.472  1.504
9          0.139 0.669  2.089
10         0.054 0.464  4.732

### 4.2.3. 주의 : nan은 groupby시 자동으로 filter out 되기 때문에, 미리 전처리 다 하는게 좋음

In [93]:
df = pd.DataFrame({
    'a':['소형주',np.nan,'대형주','대형주'],
    'b':[np.nan,2,3,np.nan]
})
df

,a,b
0,소형주,NaN
1,NaN,2.000
2,대형주,3.000
3,대형주,NaN


In [97]:
df.groupby(['a'])['b'].mean()

a
대형주   3.000
소형주     NaN
Name: b, dtype: float64